# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Machine Learning: Logistic Regression** </center>
---
**Profesor**: Pablo Camarillo Ramirez

## Recreate docker images and spark cluster:

Go to **spark** directory and run:
     
     sh build-images.sh
     docker compose up --scale spark-worker=1 -d

# Create SparkSession

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Logistic Regression") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/22 01:20:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Collect Data

In [2]:
from pcamarillor.spark_utils import SparkUtils
# Create a small dataset as a list of tuples
# Format: (label, feature_x1, feature_x2)
data = [
    (1.0, 2.0, 3.0),
    (0.0, 1.0, 2.5),
    (1.0, 3.0, 5.0),
    (0.0, 0.5, 1.0),
    (1.0, 4.0, 6.0)
]

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("label", "float"), 
                                     ("feature_x1", "float"),
                                     ("feature_x2", "float")])

# Convert list to a DataFrame
df = spark.createDataFrame(data, schema)
df.show()

+-----+----------+----------+
|label|feature_x1|feature_x2|
+-----+----------+----------+
|  1.0|       2.0|       3.0|
|  0.0|       1.0|       2.5|
|  1.0|       3.0|       5.0|
|  0.0|       0.5|       1.0|
|  1.0|       4.0|       6.0|
+-----+----------+----------+



### Assemble the features into a single vector column

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["feature_x1", "feature_x2"], outputCol="features")
data_with_features = assembler.transform(df).select("label", "features")
data_with_features.show()                                     

+-----+---------+
|label| features|
+-----+---------+
|  1.0|[2.0,3.0]|
|  0.0|[1.0,2.5]|
|  1.0|[3.0,5.0]|
|  0.0|[0.5,1.0]|
|  1.0|[4.0,6.0]|
+-----+---------+



# Data splitting
#### 80% training data and 20% testing data

In [13]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=13)

### Show dataset (for debugging)

In [14]:
print("Original Dataset")
df.show()

# Print train dataset
print("train set")
train_df.show()

Original Dataset
+-----+----------+----------+
|label|feature_x1|feature_x2|
+-----+----------+----------+
|  1.0|       2.0|       3.0|
|  0.0|       1.0|       2.5|
|  1.0|       3.0|       5.0|
|  0.0|       0.5|       1.0|
|  1.0|       4.0|       6.0|
+-----+----------+----------+

train set
+-----+---------+
|label| features|
+-----+---------+
|  0.0|[1.0,2.5]|
|  0.0|[0.5,1.0]|
|  1.0|[3.0,5.0]|
|  1.0|[4.0,6.0]|
+-----+---------+



# Create ML Model

In [15]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.2)

# Train ML Model

In [16]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

Coefficients: [0.4975285118854061,0.34768295416319633]


## Predictions

In [17]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction", "probability").show()

+---------+----------+--------------------+
| features|prediction|         probability|
+---------+----------+--------------------+
|[2.0,3.0]|       0.0|[0.56836782386618...|
+---------+----------+--------------------+



# Test ML Model

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")  

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


# Lab 11: Logistic regression To predict heart disease

In [23]:
!ls data/ml/logistic_regresion/

framingham.csv


# Data collection

In [24]:
# Define schema for the DataFrame
heart_schema = SparkUtils.generate_schema([
    ("male", "int"), 
    ("age", "int"), 
    ("education", "int"), 
    ("currentSmoker", "int"), 
    ("cigsPerDay", "int"), 
    ("BPMeds", "int"), 
    ("prevalentStroke", "int"), 
    ("prevalentHyp", "int"), 
    ("diabetes", "int"), 
    ("totChol", "int"), 
    ("sysBP", "float"), 
    ("diaBP", "float"), 
    ("BMI", "float"), 
    ("heartRate", "int"), 
    ("glucose", "int"), 
    ("TenYearCHD", "int")])

# Source: https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download

heart_df = spark.read \
                .option("header", "true") \
                .schema(heart_schema) \
                .csv("/opt/spark/work-dir/data/ml/logistic_regresion/framingham.csv")

heart_df.printSchema()
print(f"Number of records:{heart_df.count()}")

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: integer (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

Number of records:4238


In [25]:
heart_df.show(2)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|  BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|   1| 39|        4|            0|         0|     0|              0|           0|       0|    195|106.0| 70.0|26.97|       80|     77|         0|
|   0| 46|        2|            0|         0|     0|              0|           0|       0|    250|121.0| 81.0|28.73|       95|     76|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
only showing top 2 rows


25/10/22 02:11:07 ERROR TaskSchedulerImpl: Lost executor 0 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 03:57:16 ERROR TaskSchedulerImpl: Lost executor 1 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 04:24:11 ERROR TaskSchedulerImpl: Lost executor 2 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 05:16:32 ERROR TaskSchedulerImpl: Lost executor 3 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 05:32:33 ERROR TaskSchedulerImpl: Lost executor 4 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 06:14:52 ERROR TaskSchedulerImpl: Lost executor 5 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 06:30:52 ERROR TaskSchedulerImpl: Lost executor 6 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/10/22 07:17:54 ERROR TaskSchedulerImpl: Lost executor 7 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 

# Data Splitting

# Create ML Model

# Train ML Model

# Test ML Model

In [ ]:
sc.stop()